# A Simple Notebook using OpenAI for Text Summarization

In [1]:
import os
import pandas as pd 
from dotenv import load_dotenv
from openai import OpenAI
import yfinance as yf
from datetime import datetime, timedelta

Load OpenAI credentials and configure basic OpenAI instance

In [7]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
openai = OpenAI()

Let's find some good websites for financial information pertaining to the stock market

In [10]:
message = "Provide some good websites for financial information pertaining to the stock market. These should include both sites with news and analysis, as well as sites that provide data and statistics on stock performance, especially sites where basic financial information, such as EPS, revenue, earnings, etc. Please include a brief description of each site and what it offers."

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant, who is an expert financial analyst."},
        {"role": "user", "content": message}
    ]
)

In [11]:
print(response.choices[0].message.content)  # Print the response from the model

Certainly! Here’s a list of reputable websites that offer a mix of financial news, analysis, data, and statistics on stock performance.

### 1. **Yahoo Finance**
   - **Description**: Yahoo Finance is one of the most comprehensive financial news platforms. It offers stock quotes, financial news, and data on a wide range of financial metrics, such as EPS, revenue, and historical performance.
   - **Features**: Customizable watchlists, portfolio management tools, market news, and interactive charts.

### 2. **Bloomberg**
   - **Description**: Bloomberg is a leading global business and financial information and news provider. The site offers high-quality articles covering market trends, data on equities, commodities, fixed income, and extensive financial metrics.
   - **Features**: Market data analysis, Bloomberg Terminal access for professionals, and financial news updates.

### 3. **MarketWatch**
   - **Description**: MarketWatch provides a mix of business news, personal finance informa

Yahoo Finance was ranked first, so let's explore it

In [8]:
def get_stock_financials(ticker_symbol: str) -> dict:
    """
    Extract key financial metrics for a given stock ticker.
    
    Args:
        ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
    
    Returns:
        dict: Dictionary containing financial metrics
    """
    try:
        # Initialize ticker object
        ticker = yf.Ticker(ticker_symbol)
        
        # Get financial data with proper error handling
        financials = {
            'Company Info': {
                'Name': ticker.info.get('longName', 'N/A'),
                'Sector': ticker.info.get('sector', 'N/A'),
                'Industry': ticker.info.get('industry', 'N/A'),
            },
            'Key Metrics': {
                'Market Cap': ticker.info.get('marketCap', 'N/A'),
                'PE Ratio': ticker.info.get('trailingPE', 'N/A'),
                'Forward PE': ticker.info.get('forwardPE', 'N/A'),
                'Dividend Yield': ticker.info.get('dividendYield', 'N/A'),
                'EPS (TTM)': ticker.info.get('trailingEps', 'N/A'),
                'Forward EPS': ticker.info.get('forwardEps', 'N/A'),
            }
        }

        # Get income statement data
        try:
            income_stmt = ticker.income_stmt
            if income_stmt is not None:
                financials['Yearly Income Statement'] = income_stmt.to_dict()
        except:
            financials['Yearly Income Statement'] = {}

        # Get quarterly income statement
        try:
            quarterly_income = ticker.quarterly_income_stmt
            if quarterly_income is not None:
                financials['Quarterly Income Statement'] = quarterly_income.to_dict()
        except:
            financials['Quarterly Income Statement'] = {}

        # Get balance sheet data
        try:
            balance_sheet = ticker.balance_sheet
            if balance_sheet is not None:
                financials['Yearly Balance Sheet'] = balance_sheet.to_dict()
        except:
            financials['Yearly Balance Sheet'] = {}

        return financials
    
    except Exception as e:
        return {'error': f"Failed to fetch data: {str(e)}"}

def display_financial_summary(ticker_symbol: str, num_quarters: int = 20) -> None:
    """
    Display a formatted summary of key financial metrics.
    
    Args:
        ticker_symbol (str): Stock ticker symbol
        num_quarters (int): Number of past quarters to display (default: 20)
    """
    data = get_stock_financials(ticker_symbol)
    
    if 'error' in data:
        print(f"Error: {data['error']}")
        return
    
    print(f"\n=== Financial Summary for {data['Company Info']['Name']} ({ticker_symbol}) ===\n")
    
    # Company Info
    print("Company Information:")
    for key, value in data['Company Info'].items():
        print(f"{key}: {value}")
    
    print("\nKey Metrics:")
    for key, value in data['Key Metrics'].items():
        if key == 'Market Cap' and value != 'N/A':
            value = f"${value:,.0f}"
        elif key == 'Dividend Yield' and value != 'N/A':
            value = f"{value:.2%}"
        elif 'EPS' in key and value != 'N/A':
            value = f"${value:.2f}"
        print(f"{key}: {value}")
    
    # Historical quarterly results
    if data.get('Quarterly Income Statement'):
        print("\nQuarterly Results (Last 20 Quarters):")
        print("\nDate       Revenue        Net Income")
        print("-" * 45)
        
        quarters = list(data['Quarterly Income Statement'].keys())[:num_quarters]
        
        for quarter_date in quarters:
            quarterly_data = data['Quarterly Income Statement'][quarter_date]
            revenue = quarterly_data.get('Total Revenue', 'N/A')
            net_income = quarterly_data.get('Net Income', 'N/A')
            
            # Format date - using proper date formatting
            if isinstance(quarter_date, datetime):
                date_str = quarter_date.strftime("%Y-%m")
            else:
                date_str = str(quarter_date)[:7]  # Get YYYY-MM format
            
            # Format numbers
            revenue_str = f"${revenue:,.0f}" if isinstance(revenue, (int, float)) else "N/A"
            net_income_str = f"${net_income:,.0f}" if isinstance(net_income, (int, float)) else "N/A"
            
            print(f"{date_str:<10} {revenue_str:<14} {net_income_str:<14}")



In [9]:
# Example usage
ticker = "AAPL"  # or any other stock symbol
display_financial_summary(ticker)


=== Financial Summary for Apple Inc. (AAPL) ===

Company Information:
Name: Apple Inc.
Sector: Technology
Industry: Consumer Electronics

Key Metrics:
Market Cap: $3,136,816,676,864
PE Ratio: 32.66252
Forward PE: 25.273165
Dividend Yield: 51.00%
EPS (TTM): $6.43
Forward EPS: $8.31

Quarterly Results (Last 20 Quarters):

Date       Revenue        Net Income
---------------------------------------------
2025-03    $95,359,000,000 $24,780,000,000
2024-12    $124,300,000,000 $36,330,000,000
2024-09    $94,930,000,000 $14,736,000,000
2024-06    $85,777,000,000 $21,448,000,000
2024-03    $90,753,000,000 $23,636,000,000


In [10]:
def analyze_available_quarters(ticker_symbol: str) -> None:
    """
    Analyze and display the available quarterly data for a given ticker.
    """
    ticker = yf.Ticker(ticker_symbol)
    quarterly_data = ticker.quarterly_income_stmt
    
    if quarterly_data is not None:
        num_quarters = len(quarterly_data.columns)
        print(f"\nAvailable quarterly data for {ticker_symbol}:")
        print(f"Number of quarters: {num_quarters}")
        print("\nDate range:")
        print(f"Earliest: {quarterly_data.columns[-1].strftime('%Y-%m-%d')}")
        print(f"Latest: {quarterly_data.columns[0].strftime('%Y-%m-%d')}")
    else:
        print("No quarterly data available")

# Test with Apple stock
ticker = "AAPL"
analyze_available_quarters(ticker)


Available quarterly data for AAPL:
Number of quarters: 5

Date range:
Earliest: 2024-03-31
Latest: 2025-03-31
